### 首先是数据集
MNIST 手写数据集太老了，可以选择Fashion-MNIST 数据集

In [ ]:
%matplotlib inline
import torch
import torchvision
from torch.utils import data
import matplotlib.pyplot as plt
from torchvision import transforms
import numpy as np
from d2l import torch as d2l

d2l.use_svg_display() # 使用svg来显示图片

In [3]:
print('hello world')

hello world
